<a href="https://colab.research.google.com/github/Edwin372/Thesis/blob/main/Vudenc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/LauraWartschinski/VulnerabilityDetection.git


fatal: destination path 'VulnerabilityDetection' already exists and is not an empty directory.


In [4]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!mv /content/VulnerabilityDetection/Code/w2v /content
!mv /content/VulnerabilityDetection/Code/myutils.py /content
!mv /content/VulnerabilityDetection/Code/data /content

mv: cannot stat '/content/VulnerabilityDetection/Code/w2v': No such file or directory
mv: cannot stat '/content/VulnerabilityDetection/Code/myutils.py': No such file or directory
mv: cannot stat '/content/VulnerabilityDetection/Code/data': No such file or directory


In [6]:
!unrar e /content/w2v/othermodels.rar /content/w2v


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/w2v/othermodels.rar


Would you like to replace the existing file /content/w2v/word2vec_withoutString10-1-200.model
6635437 bytes, modified on 2019-11-06 08:31
with a new one
6635437 bytes, modified on 2019-11-06 08:31

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit A

Extracting  /content/w2v/word2vec_withoutString10-1-200.model              0%  1%  2%  3%  OK 
Extracting  /content/w2v/word2vec_withoutString10-5-200.model              3%  4%  5%  6%  OK 
Extracting  /content/w2v/word2vec_withoutString10-10-200.model             6%  7%  8%  9% 10%  OK 
Extracting  /content/w2v/word2vec_withoutString10-30-200.model            10% 11% 12% 13%  OK 
Extracting  /content/w2v/word2vec_withoutString10-50-200.model            13% 14% 15% 16%  OK 
Extracting  /content/w2v/word2vec_withoutString10

In [7]:
!unrar e /content/w2v/pythontraining_withString.rar /content/w2v


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/w2v/pythontraining_withString.rar


Would you like to replace the existing file /content/w2v/pythontraining_withString_X
337300692 bytes, modified on 2019-10-23 00:41
with a new one
337300692 bytes, modified on 2019-10-23 00:41

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit A

Extracting  /content/w2v/pythontraining_withString_X                       0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 62% 63% 64% 65% 66% 67% 68%

In [8]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import torch
import torch.nn as nn
import myutils
import sys
import os
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score 
from sklearn.metrics import f1_score 
from sklearn.utils import class_weight
from gensim.models import Word2Vec 
import wandb

use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
#paramerters for filtering and creation of samples
mode='sql'
progress = 0
count = 0
restriction = [20000, 5, 6, 10]
step = 5
full_length = 200
mode_param = str(step)+"_"+str(full_length)

In [12]:
#parameters for w2v model
mincount = 5000 #minimum time a word has to appear in the corpus to be in the w2vec model
iterationen = 300  #training iterations for the w2v model
s = 200 #d
w = "withString"
w2v_name = "word2vec_" + w + str(mincount) + "-" + str(iterationen) + "-" + str(s)
w2v_model = 'w2v/'+ w2v_name + ".model"

w2v_model = Word2Vec.load(w2v_model)


In [13]:
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: eddiechen372. Use `wandb login --relogin` to force relogin


True

In [14]:
# import json

all_blocks = []
# with open('data/plain_' + mode, 'r') as infile:
  # data = json.load(infile)
# allblocks = []
# for r in data:
#   progress = progress + 1
#   for c in data[r]:
    
#     if "files" in data[r][c]:                      
#       # if len(data[r][c]["files"]) > restriction[3]:
#       #   # too many files
#       #  continue
      
#       for f in data[r][c]["files"]:
        
#         # if len(data[r][c]["files"][f]["changes"]) >= restriction[2]:
#         #   #too many changes in a single file
#         #   continue
        
#         if not "source" in data[r][c]["files"][f]:
#           #no sourcecode
#           continue
        
#         if "source" in data[r][c]["files"][f]:
#           sourcecode = data[r][c]["files"][f]["source"]                          
#           # if len(sourcecode) > restriction[0]:
#           #   #sourcecode is too long
#           #   continue
          
#           allbadparts = []
          
#           for change in data[r][c]["files"][f]["changes"]:
#                 #get the modified or removed parts from each change that happened in the commit                  
#                 badparts = change["badparts"]
#                 count = count + len(badparts)
#                 # if len(badparts) > restriction[1]:
#                 #   #too many modifications in one change
#                 #   break
                 
                
#                 for bad in badparts:
#                   #check if they can be found within the file
#                   pos = myutils.findposition(bad,sourcecode)
#                   if not -1 in pos:
#                       allbadparts.append(bad)
#                 # if (len(allbadparts) > restriction[4]):
#                 #   # too many bad positions in the file
#                 #   break
#           if(len(allbadparts) > 0):
#             # if len(allbadparts) < restriction[2]:
#               # find the positions of all modified parts
#               positions = myutils.findpositions(allbadparts,sourcecode)

#               #get the file split up in samples
#               blocks = myutils.getblocks(sourcecode, positions, step, full_length)
              
#               for b in blocks:
#                   #each is a tuple of code and label
#                   allblocks.append(b)


In [15]:
import torch
from torch.utils.data import Dataset
from torchtext.transforms import ToTensor, PadTransform
import random
class MyDataset(Dataset):
  def __init__(self, allblocks ,data_split='train', transform=None, target_transform=None):
    self.allblocks = allblocks 
    self.transform = transform
    self.target_transform = target_transform
    self.data_split=data_split
  
    keys = []
    for i in range(len(self.allblocks)):
      keys.append(i)
    random.shuffle(keys)
    cutoff = round(0.7 * len(keys)) #     70% for the training set
    cutoff2 = round(0.85 * len(keys)) #   15% for the validation set and 15% for the final test set
    self.keystrain = keys[:cutoff]
    self.keysvalid = keys[cutoff:cutoff2]
    self.keystest = keys[cutoff2:]

    default_idx = self.keystrain[0]
    block = self.allblocks[default_idx]
    code = block[0]
    self.default_token = myutils.getTokens(code)

  def __len__(self):
    if self.data_split == 'train': 
      return len(self.keystrain)
    elif self.data_split == 'valid':
      return len(self.keysvalid)
    elif self.data_split == 'test': 
      return len(self.keystest)
    else:
      print('Invalid split')

  def __getitem__(self, idx):
    word_vectors = w2v_model.wv
    if self.data_split == 'train': 
      key_idx = self.keystrain[idx]
    elif self.data_split == 'valid':
      key_idx = self.keysvalid[idx]
    elif self.data_split == 'test':
      key_idx = self.keystest[idx]
    block = self.allblocks[key_idx]
    code = block[0]
    token = myutils.getTokens(code) #get all single tokens from the snippet of code
    if(len(token) == 0): 
      print(len(token))
      token = self.default_token

    vectorlist = []
    if block[1] == 1: 
      label = 0 
    else: 
      label = 1
    for t in token: #convert all tokens into their word2vec vector representation
      if t in word_vectors.vocab and t != " ":
        word_vector = w2v_model[t]
        vectorlist.append(word_vector) 
    vectorlist = torch.tensor(vectorlist)
    if self.transform:
      vectorlist = self.transform(vectorlist)
    if self.target_transform:
      label = self.target_transform(label)
    return vectorlist, label 


In [25]:
# from torchtext.transforms import ToTensor, PadTransform

# train_dataset = MyDataset(
#     allblocks=allblocks,
#     data_split='train',
# )
# valid_dataset = MyDataset(
#     allblocks=allblocks,
#     data_split='valid'
# )
# test_dataset = MyDataset(
#     allblocks=allblocks,
#     data_split='test'
# )


In [26]:
# data_path = '/content/drive/MyDrive/data'
# isFiltered = False 
# name = '_filtered' if isFiltered else '' 
# torch.save(train_dataset, f'{data_path}/train_data_{mode}{name}.pth')
# torch.save(valid_dataset, f'{data_path}/valid_data_{mode}{name}.pth')
# torch.save(test_dataset, f'{data_path}/test_data_{mode}{name}.pth')

In [29]:
# train_dataset = torch.load(f'{data_path}/train_data_{mode}{name}.pth')
# valid_dataset = torch.load(f'{data_path}/valid_data_{mode}{name}.pth')
# test_dataset = torch.load(f'{data_path}/test_data_{mode}{name}.pth')

In [16]:
from torch.nn.utils.rnn import pad_sequence

class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx
        
    
    def __call__(self, batch):
      source = []
      target = []
      for i in range(len(batch)):
        if batch[i][0].shape[0] != 0:
          source.append(batch[i][0]) 
          target.append(batch[i][1])

      for i in range(len(batch) - len(source)):
        source.append(source[-1])
        target.append(target[-1])
      
      source = pad_sequence(source, batch_first=True, padding_value = self.pad_idx) 
      return source, target 

In [17]:
from torch.utils.data import DataLoader 

def get_data_loader(dataset, batch_size, num_workers=0, shuffle=True, pin_memory=True): #increase num_workers according to CPU
    pad_idx = 0 
    loader = DataLoader(
        dataset,
        batch_size = batch_size, 
        num_workers = num_workers,
        shuffle=shuffle,
        pin_memory=pin_memory, 
        collate_fn = MyCollate(pad_idx=pad_idx)
    ) 
    return loader

In [18]:
 
# train_loader = get_data_loader(train_dataset, batch_size=batch_size )
# valid_loader = get_data_loader(valid_dataset, batch_size=batch_size)
# test_loader = get_data_loader(test_dataset, batch_size=batch_size)

In [19]:
class MyGRU(nn.Module):
  def __init__(self, hidden_size=200, num_layers=1, bidirectional=False, dropout=0.2):
    super(MyGRU, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = w2v_model
    self.num_layers = num_layers
    self.bidirectional = bidirectional
    self.gru = nn.GRU(
        hidden_size,
        hidden_size,
        dropout=dropout,
        batch_first=True, 
        num_layers=self.num_layers,
        bidirectional=self.bidirectional
    )
    self.out = nn.Linear(hidden_size, 1)
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, input, hidden):
    hidden = hidden.cuda()
    output, hidden = self.gru(input, hidden )
    output = self.out(output)
    output = self.sigmoid(output)
    return output, hidden
  
  def initHidden(self, batch_size=1):
    hidden = torch.zeros(2 if self.bidirectional else 1 * self.num_layers, batch_size ,self.hidden_size).to(device)
    return hidden 
    

In [20]:
class MyLSTM(nn.Module):
  def __init__(self, hidden_size=200, num_layers=1, bidirectional=False, dropout=0.2):
    super(MyLSTM, self).__init__()
    self.hidden_size = hidden_size
    self.embedding = w2v_model
    self.num_layers = num_layers
    self.bidirectional = bidirectional
    self.lstm = nn.LSTM(
        hidden_size,
        hidden_size,
        dropout=dropout,
        batch_first=True, 
        num_layers=self.num_layers,
        bidirectional=self.bidirectional
    )
    self.out = nn.Linear(hidden_size, 1)
    self.sigmoid = nn.Sigmoid()
  
  def forward(self, input, hidden):
    output, hidden = self.lstm(input, hidden)
    output = self.out(output)
    output = self.sigmoid(output)
    return output, hidden
  
  def initHidden(self, batch_size=1):
    h0 = torch.zeros(2 if self.bidirectional else 1 * self.num_layers, batch_size ,self.hidden_size).to(device)
    c0 = torch.zeros(2 if self.bidirectional else 1 * self.num_layers, batch_size ,self.hidden_size).to(device)
    return h0, c0 
    

In [21]:
import math
import time
print_every = 1000
plot_every = 50000

def timeSince(since):
  now = time.time()
  s = now - since
  m = math.floor(s/60)
  s -= m * 60
  return f'{m}m {math.floor(s)}s' 


In [22]:
def weighted_binary_cross_entropy(output, target, weights=None):
    # print(output, target)
    if weights is not None:
        assert len(weights) == 2
        loss = weights[1] * (target * torch.log(output)) + \
               weights[0] * ((1 - target) * torch.log(1 - output))
    else:
        loss = target * torch.log(output) + (1 - target) * torch.log(1 - output)

    return torch.neg(torch.mean(loss))

In [23]:

from torchmetrics import F1Score, Precision, Recall

def evaluate(model, loader, batch_size, class_weights):
  targets = torch.tensor([]).cuda() 
  preds = torch.tensor([]).cuda() 
  start = time.time()
  all_losses = []

  
  with torch.no_grad():
    print_count = 0 
    for iter, (input , labels) in enumerate(loader):

      hidden = model.initHidden(batch_size=batch_size)
      label_tensor = torch.tensor(labels).to(device)
      input = input.to(device)

      targets = torch.cat((targets, torch.tensor(label_tensor)))
      output, hidden = model(input, hidden)
      final_output = output[:, -1, :].T

      loss = weighted_binary_cross_entropy(final_output , label_tensor, weights=class_weights)
      all_losses.append(loss)
      preds = torch.cat((preds, final_output), 1)

    eval_loss = torch.mean(torch.tensor(all_losses))
    preds = torch.squeeze(preds)
    targets = targets.int()
    
  preds = torch.tensor(preds).to(device)
  targets = torch.tensor(targets).to(device)
  f1 = F1Score(num_classes=1).to(device)

  precision = Precision(num_classes=1).to(device)
  recall = Recall(num_classes=1).to(device)

  f1_score = f1(preds, targets)
  precision_score = precision(preds, targets)
  recall_score = recall(preds, targets)

  wandb.log({
    'f1 score': f1_score,
    'precision score': precision_score,
    'recall score': recall_score,
    'eval loss': eval_loss
  })


In [24]:
# evaluate(model)

In [25]:
import torch.optim as optim
def train_mini_batch(model, input_tensor, label_tensor, batch_size, optimizer, class_weights):
  hidden = model.initHidden(batch_size=batch_size)
  model.zero_grad()
  label_tensor = torch.tensor([label_tensor]).to(device)
  input = input_tensor.to(device)
  output, hidden = model(input, hidden)
  final_output = output[:, -1, :].T
  loss = weighted_binary_cross_entropy(final_output , label_tensor, weights=class_weights)
  loss.backward()
  optimizer.step()
  return final_output, loss.item()
  

In [27]:
def train(model, loader, batch_size, optimizer, class_weights):
  all_losses = []
  for iter, (input_tensor, label_tensor) in enumerate(loader):
    preds, loss = train_mini_batch(model, input_tensor, label_tensor, batch_size, optimizer, class_weights)
    all_losses.append()
    
  wandb.log({'train_loss': torch.mean(torch.tensor(all_losses))})


In [68]:
def getModel(architecture):
  if architecture == 'GRU':
    return MyGRU().to(device)
  else:
    return MyLSTM().to(device) 

In [69]:
model = getModel('LSTM') 

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [58]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [77]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
datasets = ['sql', 'xss', 'xsrf', 'path_disclosure', 'open_redirect', 'command_injection', 'remote_code_execution']
architectures = ['LSTM', 'GRU']
data_path = '/content/drive/MyDrive/data'
def run():
  conf = dict(
    learning_rate = 0.0005,
    infra = 'Google colab',
    dataset = '',
    architecure = '',
    log_interval=1000,
    epochs = 10,
    batch_size=128, 
    full_length = 200,
    step = 5,
    word_min_count = 5000,
    iterationen = 300,
    hidden_size = 200,
    with_string = 'withString',
    seed = 42,
  )

  for dataset in datasets:   
    print(f'current dataset: {dataset}')
    conf['dataset'] = dataset

    train_dataset = torch.load(f'{data_path}/train_data_{conf["dataset"]}.pth')
    train_loader = get_data_loader(train_dataset, batch_size=conf['batch_size'])

    valid_dataset = torch.load(f'{data_path}/valid_data_{conf["dataset"]}.pth')
    valid_loader = get_data_loader(valid_dataset, batch_size=conf['batch_size'])

    # test_dataset = torch.load(f'{data_path}/test_data_{conf["dataset"]}.pth')
    # test_loader = get_data_loader(test_dataset, batch_size=conf['batch_size'])


    print('Calulating class weigths...')
    y = np.concatenate([label for item, label in train_loader])
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
    print(f'{dataset} dataset class weights: {class_weights}')

    for architecture in architectures:
      print(f'current architecture: {architecture}')
      conf['architecture'] = architecture
      wandb.init(
        project='Thesis',
        notes=f'Tweak baseline' ,
        tags=[architecture],
        config=conf

      )
      model = getModel(architecture)
      optimizer = optim.Adam(model.parameters(), lr=conf['learning_rate'],  weight_decay=0)
      wandb.watch(model, log='all')

      for i in range(conf['epochs']):
        train(model, train_loader , conf.batch_size, optimizer, class_weights)
        evaluate(model, valid_loader, conf.batch_size, class_weights)
      


In [75]:
run()

current dataset: sql
Calulating class weigths...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


sql dataset class weights: [0.56109203 4.59218705]
current architecture: LSTM


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


AttributeError: ignored

In [79]:
train_dataset = torch.load(f'{data_path}/train_data_sql.pth')
train_loader = get_data_loader(train_dataset, batch_size=128)
label_batches = [label for item, label in train_loader]

y = np.concatenate(label_batches)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


KeyboardInterrupt: ignored

In [2]:
array = []
train_loader = get_data_loader(train_dataset, batch_size=128)
for item, label in train_loader:
  
  print(label.shape)
  array.append(label)


NameError: ignored